# CT-Rate


In [10]:
from dicom_helpers import nifti_to_dicom, add_metadata_to_series, add_metadata_to_study

## Example 1
- Calcification
- Atelectasis
- Opacity
- Consolidation

### Original


In [ ]:

findings = "Multiple venous collaterals are present in the anterior left chest wall and are associated with the anterior jugular vein at the level of the right sternoclavicular junction. Left subclavian vein collapsed (chronic occlusion pathology?). Trachea, both main bronchi are open. Calcific plaques are observed in the aortic arch. Other mediastinal main vascular structures, heart contour, size are normal. Thoracic aorta diameter is normal. Pericardial effusion-thickening was not observed. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. No enlarged lymph nodes in prevascular, pre-paratracheal, subcarinal or bilateral hilar-axillary pathological dimensions were detected. When examined in the lung parenchyma window; Linear atelectasis is present in both lung parenchyma. Subsegmental atelectasis is observed in the right middle lobe. Thickening of the bronchial wall and peribronchial budding tree-like reticulonodular densities are observed in the bilateral lower lobes. Peribronchial minimal consolidation is seen in the lower lobes in places. The findings were evaluated primarily in favor of the infectious process. The left kidney partially entering the section is atrophic. The right kidney could not be evaluated because it did not enter the section. Other upper abdominal organs included in the sections are normal. No space-occupying lesion was detected in the liver that entered the cross-sectional area. Bilateral adrenal glands were normal and no space-occupying lesion was detected. There are osteophytes with anterior extension in the thoracic vertebrae."
impressions="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Calcification, Atelectasis, Opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_1_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file,
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.1',
               series_instance_uid='1.1.1',
               study_id='1', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
               )

add_metadata_to_study('1.1', findings, 'Findings')
add_metadata_to_study('1.1', impressions, 'Impressions')

### Generated

In [ ]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Calcification, Atelectasis, Opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.2',
               study_id='2', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
add_metadata_to_series('1.1.2', prompt, 'SeriesPrompt')

In [ ]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Calcification, Atelectasis, Opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.3',
               study_id='3', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)

add_metadata_to_series('1.1.3', prompt, 'SeriesPrompt')

In [ ]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Calcification, Atelectasis, Opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_sample_2.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 3", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.4',
               study_id='4', 
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
add_metadata_to_series('1.1.4', prompt, 'SeriesPrompt')

### Counterfactual

In [ ]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  No thickening of the bronchial wall.  Peribronchial reticulonodular densities in the lower lobes, minimal consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Calcification, Atelectasis, Opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="No Thickening (counterfactual 1)", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.5',
               study_id='5',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
add_metadata_to_series('1.1.5', prompt, 'SeriesPrompt')

In [ ]:
prompt="Multiple venous collaterals in the anterior left chest wall and collapsed appearance in the left subclavian vein (chronic occlusion?).  Thickening of the bronchial wall in both lungs.  Peribronchial reticulonodular densities in the lower lobes, large consolidations (infection process?).  Atelectasis in both lungs.  Thoracic spondylosis."
description="Calcification, Atelectasis, Opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1_a_1_counterf_2_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Thickening (counterfactual 2)", 
               modality = "AI",
               study_instance_uid='1.1',
               series_instance_uid='1.1.6',
               study_id='6',
               patient_name="Example Patient 1",
               patient_id="train_1_a_1",
               description=description,
)
add_metadata_to_series('1.1.6', prompt, 'SeriesPrompt')

## Example 2
- Calcification
- Lymphadenopathy
- Nodule
- Opacity


### Original 

In [ ]:

findings = "Trachea, both main bronchi are open. Mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Calcific plaques are present in the coronary arteries. Thoracic esophagus calibration was normal and no significant pathological wall thickening was detected. Millimetric lymph nodes with a short axis not exceeding 1 cm are observed in the mediastinum. When examined in the lung parenchyma window; Widespread ground glass densities are observed in both lung parenchyma showing a tendency to central and peripheral fusion. A 5 mm calcific nodule was observed in the posterobasal region of the lower lobe of the right lung. A millimetric hypodense lesion was observed in segment 2 of the liver in the upper abdominal organs included in the sections. Bone structures in the study area are natural. Osteodegenerative changes are observed in the vertebrae."
impressions="Findings compatible with Covid pneumonia  Aortic and coronary artery atherosclerosis  Hypodense lesion (cyst?) in liver segment 2"
description= "Calcification, Lymphadenopathy, Nodule, Opacity"
nifti_file = '../data/nifti/ct-rate/train/train_10_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.2',
               series_instance_uid='1.2.1',
               study_id='1',
               patient_name="Example Patient 2",
               patient_id="train_10_a_1",
               description= description,
               )
add_metadata_to_study('1.2', findings, 'Findings')
add_metadata_to_study('1.2', impressions, 'Impressions')

### Generated

In [ ]:
prompt=" Findings compatible with Covid pneumonia  Aortic and coronary artery atherosclerosis  Hypodense lesion (cyst?) in liver segment 2"
description= "Calcification, Lymphadenopathy, Nodule, Opacity"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_10_a_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.2',
               series_instance_uid='1.2.2',
               study_id='2',
               patient_name="Example Patient 2",
               patient_id="train_10_a_1",
               description= description,

               )
add_metadata_to_series('1.2.2', prompt, 'SeriesPrompt')

# Example 3
- Effusion
- Emphysema
- Atelectasis
- Lung nodule
- Consolidation


### Original 

In [ ]:

findings = "A central venous catheter inserted from the right was observed. Trachea and both main bronchi are open and no obstructive pathology is detected. Mediastinal vascular structures and cardiac examination could not be evaluated optimally due to the lack of contrast. The pulmonary trunk caliber was measured at 30 mm and was wider than normal. Heart contour, the size is natural. There is minimal pericardial effusion. The effusion measuring approximately 80 mm in size is observed on the right. There is an area of increased density in the lung parenchyma adjacent to the effusion, which is considered secondary to compressive atelectasis. No pathological increase in wall thickness was observed in the thoracic esophagus. In the mediastinum, in both axillary regions and in the supraclavicular fossa, no lymph nodes were observed in pathological size and appearance. In the lower lobe of the left lung, there is an area of increase in density consistent with the consolidation observed in the air bronchograms. Although the appearance may be secondary to atelectasis, underlying pneumonic infiltration cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings. No mass lesion was observed in both lungs. There are a few millimetric nodules in the left lung, the largest of which is 5.5x3.5 mm in the upper lobe inferior lingular segment. Emphysematous changes were observed in both lungs. There are findings consistent with peritoneal carcinomatosis in the upper abdominal sections within the image, and hypodense lesions in the liver and spleen parenchyma that cannot be characterized in this examination. No lytic or destructive lesions were detected in the bone structures within the image."
impressions="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings.  A few millimetric nodules in the left lung."
description= "Effusion, Emphysema, Atelectasis, Lung nodule, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_100_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.3',
               series_instance_uid='1.3.1',
               study_id='1',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )
add_metadata_to_study('1.3', findings, 'Findings')
add_metadata_to_study('1.3', impressions, 'Impressions')

### Generated

In [ ]:
prompt="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings.  A few millimetric nodules in the left lung."
description= "Effusion, Emphysema, Atelectasis, Lung nodule, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_100_a_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.3',
               series_instance_uid='1.3.2',
               study_id='2',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )
add_metadata_to_series('1.3.2', prompt, 'SeriesPrompt')

### Counterfactual

In [ ]:
prompt="Right pleural effusion, area of increased density in the adjacent lung parenchyma evaluated in favor of compressive atelectasis.  Density increase area in the lower lobe of the left lung consistent with the consolidation observed in air bronchograms; Pneumonic infiltration, which may be related to atelectasis, cannot be excluded. It is recommended to be evaluated together with clinical and laboratory findings."
description= "Effusion, Emphysema, Atelectasis, Lung nodule, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_100_a_1_counterf_1_sample_0.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.3',
               series_instance_uid='1.3.3',
               study_id='3',
               patient_name="Example Patient 3",
               patient_id="train_100_a_1",
               description= description,
               )
add_metadata_to_series('1.3.3', prompt, 'SeriesPrompt')

# Example 4
- Lung nodule


### Original 

In [ ]:
findings = "No lymph node was observed in the supraclavicular fossa and axilla in pathological size and appearance. Thyroid gland, left lobe and isthmus are operated. Heart dimensions and compartments appear natural. Pericardial effusion was not detected. Calibrations of mediastinal major vascular structures are natural. No pneumonic infiltration or consolidation area was detected in the lung parenchyma. No suspicious mass or nodular space-occupying lesion was observed in the lung parenchyma. There are a few nonspecific nodules less than 5 mm in diameter. In the evaluation of the upper abdominal organs, including the cross-section, there is a decrease in both kidney sizes and parenchyma thickness. No lytic-destructive lesions were detected in bone structures."
impressions="Thoracic CT examination within normal limits. Decrease in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/train/train_1000_a_1.nii.gz'


nifti_to_dicom(nifti_file=nifti_file,
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.4',
               series_instance_uid='1.4.1',
               study_id='1',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description
               )
add_metadata_to_study('1.4', findings, 'Findings')
add_metadata_to_study('1.4', impressions, 'Impressions')

## Generated

In [ ]:
prompt="Thoracic CT examination within normal limits. Decrease in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1000_a_1_sample_0.nii.gz'
output_folder = '../data/nifti/ct-rate/train/train_1000_a_1/generated/'

nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.4',
               series_instance_uid='1.4.2',
               study_id='2',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description,
               )
add_metadata_to_series('1.4.2', prompt, 'SeriesPrompt')

### Counterfactual

In [ ]:
prompt="Thoracic CT examination within normal limits. Increase in both kidney sizes and parenchyma thickness. Partially thyroidectomized"
description= "Lung nodule"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1000_a_1_counterf_1_sample_0.nii.gz'


nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.4',
               series_instance_uid='1.4.3',
               study_id='3',
               patient_name="Example Patient 4",
               patient_id="train_1000_a_1",
               description= description,
               )
add_metadata_to_series('1.4.3', prompt, 'SeriesPrompt')

# Example 5
- Calcification
- Cardiomegaly
- Pericardial effusion
- Atelectasis
- Pleural effusion

### Original

In [ ]:
findings = "The examination of the patient was evaluated by comparing it with the previous pulmonary CT angiography examination. The cardiothoracic ratio increased in favor of the heart. Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta. Trachea and both main bronchi are open. No occlusive pathology was detected in the trachea and both main bronchi. No pathologically enlarged lymph nodes were detected in the mediastinum and hilar regions.3.2021. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. In the upper abdominal organs within the sections, no mass with distinguishable borders was detected as far as it can be observed within the borders of non-enhanced CT. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/train/train_1001_a_1.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.5',
               series_instance_uid='1.5.1',
               study_id='1',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description= description,
               )
add_metadata_to_study('1.5', findings, 'Findings')
add_metadata_to_study('1.5', impressions, 'Impressions')


### Generated

In [ ]:
prompt="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.5',
               series_instance_uid='1.5.2',
               study_id='2',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )
add_metadata_to_series('1.5.2', prompt, 'SeriesPrompt')

### Counterfactual

In [ ]:
prompt="Cardiomegaly, no pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.5',
               series_instance_uid='1.5.3',
               study_id='3',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )
add_metadata_to_series('1.5.3', prompt, 'SeriesPrompt')

In [ ]:
prompt="Arterial wall calcification, Coronary artery wall calcification, Hiatal hernia, Atelectasis, Lung opacity, Pulmonary fibrotic sequela, Peribronchial thickening, Consolidation, Bronchiectasis"
description= "Calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_counterf_2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file,
               series_description="Generated, 3 (counterfactual)", 
               modality = "AI",
               study_instance_uid='1.5',
               series_instance_uid='1.5.4',
               study_id='4',
               patient_name="Example Patient 5",
               patient_id="train_1001_a_1",
               description=description,
               )
add_metadata_to_series('1.5.4', prompt, 'SeriesPrompt')

# Example 6
- Calcification
- Atelectasis
- Lung opacity
- Consolidation

### Original

In [ ]:
findings = "Trachea was in the midline of both main bronchi and no obstructive pathology was observed in the lumen. The mediastinum could not be evaluated optimally in the non-contrast examination. As far as can be seen; mediastinal main vascular structures, heart contour, size are normal. Pericardial effusion-thickening was not observed. Diffuse atherosclerotic wall calcifications were observed in the thoracic aorta and coronary arteries. Thoracic esophagus calibration was normal and no significant tumoral wall thickening was detected. Sliding type hiatal hernia was observed at the lower end of the esophagus. No pathologically enlarged lymph nodes were detected in the mediastinum. No lymph node was observed in pathological size and appearance in the supraclavicular and axillary fossa. When examined in the lung parenchyma window; In both lungs, ground-glass densities including central-peripheral, crazy paving pattern and consolidation areas showing signs of vascular enlargement were observed. The outlook is consistent with Covid-19 pneumonia. The right hemidiaphragm is elevated. Diffuse linear subsegmental atelectatic changes were observed in the middle and lower lobes of the right lung. There are segmental-subsegmental bronchiectatic changes and peribronchial thickening in both lungs. Sequelae thickening was observed in the posterocostal pleura in both hemithorax. No mass lesion with distinguishable borders was detected in both lungs. A hypodense lesion area of 21x20 mm was observed at the junction of segment 4A-8 at the level of the liver dome and could not be characterized in this examination. The gallbladder was not observed (operated). The right adrenal gland and right kidney were not observed (operated). Diffuse thickening was observed in the left adrenal gland. Within the sections, free fluid-pathological lymph node was not observed in the abdomen. Mild dextroscoliosis with left opening was observed at the thoracic level. At mid-thoracic level, a bridging spur formation is observed in the right anterolateral corners of the vertebrae. Vertebral corpus heights are preserved."
impressions="Diffuse atherosclerotic wall calcifications in the thoracic aorta and coronary arteries.  Hiatal hernia.  Findings consistent with Covid-19 pneumonia in the lung parenchyma.  Elevation in the right hemidiaphragm, linear subsegmental atelectatic changes in the middle and lower lobes of the right lung.  Segmentary-subsegmentary tubular bronchiectasis, peribronchial thickening in both lungs.  Sequela thickening of posterocostal pleura in both hemithorax.  Hypodense lesion at the level of the liver dome (segment 4A-8 junction), which cannot be characterized in this examination.  Diffuse thickening of the left adrenal gland.  Spur formations bridging each other at the mid-thoracic level and mild dextroscoliosis with a secondary left-facing opening"
description="Calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/train/train_101_a_1.nii.gz'
nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='1.6',
               series_instance_uid='1.6.1',
               study_id='1',
               patient_name="Example Patient 6",
               patient_id="train_101_a_1",
               description=description,
               )
add_metadata_to_study('1.6', findings, 'Findings')
add_metadata_to_study('1.6', impressions, 'Impressions')

### Generated

In [ ]:
prompt="Diffuse atherosclerotic wall calcifications in the thoracic aorta and coronary arteries.  Hiatal hernia.  Findings consistent with Covid-19 pneumonia in the lung parenchyma.  Elevation in the right hemidiaphragm, linear subsegmental atelectatic changes in the middle and lower lobes of the right lung.  Segmentary-subsegmentary tubular bronchiectasis, peribronchial thickening in both lungs.  Sequela thickening of posterocostal pleura in both hemithorax.  Hypodense lesion at the level of the liver dome (segment 4A-8 junction), which cannot be characterized in this examination.  Diffuse thickening of the left adrenal gland.  Spur formations bridging each other at the mid-thoracic level and mild dextroscoliosis with a secondary left-facing opening"
description="Calcification, Atelectasis, Lung opacity, Consolidation"
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_101_a_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='1.6',
               series_instance_uid='1.6.2',
               study_id='2',
               patient_name="Example Patient 6",
               patient_id="train_101_a_1",
               description=description,

               )

add_metadata_to_series('1.6.2', prompt, 'SeriesPrompt')

# MedSyn Paper Examples

In [ ]:
prompt="There is extensive consolidation seen. No evidence of cardiomegaly. No evidence of pleural effusion."
description="extensive consolidation v2"
nifti_file = '../data/nifti/MedSynExamples/extensive_consolidation_v2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.1',
               series_instance_uid='2.1.1',
               study_id='1',
               patient_name="MedSyn 1",
               patient_id="2_1",
               description=description,

               )
add_metadata_to_series('2.1.1', prompt, 'SeriesPrompt')

In [ ]:
prompt="There are large pleural effusions seen. There is no airspace opacity or pneumothorax. There is no evidence of suspicious pulmonary nodule or mass."
description="large effusion v1"
nifti_file = '../data/nifti/MedSynExamples/large_effusion_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.2',
               series_instance_uid='2.2.1',
               study_id='1',
               patient_name="MedSyn 2",
               patient_id="2_2",
               description=description,

               )
add_metadata_to_series('2.2.1', prompt, 'SeriesPrompt')

In [ ]:
prompt="There are large pleural effusions seen. No evidence of cardiomegaly. No evidence of consolidation."
description="large effusion v2"
nifti_file = '../data/nifti/MedSynExamples/large_effusion_v2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.3',
               series_instance_uid='2.3.1',
               study_id='1',
               patient_name="MedSyn 3",
               patient_id="2_3",
               description=description,

               )
add_metadata_to_series('2.3.1', prompt, 'SeriesPrompt')

In [ ]:
prompt="There are large pleural effusions seen. Moderate cardiomegaly. No evidence of consolidation."
description="large effusion v2"
nifti_file = '../data/nifti/MedSynExamples/large_effusion_v2_counterfactual_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 2 (counterfactual)", 
               modality = "AI",
               study_instance_uid='2.3',
               series_instance_uid='2.3.2',
               study_id='2',
               patient_name="MedSyn 3",
               patient_id="2_3",
               description=description,

               )
add_metadata_to_series('2.3.2', prompt, 'SeriesPrompt')


In [ ]:
prompt="There are large pleural effusions seen. No evidence of cardiomegaly. There is extensive consolidation seen."
description="large effusion v2"
nifti_file = '../data/nifti/MedSynExamples/large_effusion_v2_counterfactual_2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 3", 
               modality = "AI",
               study_instance_uid='2.3',
               series_instance_uid='2.3.3',
               study_id='3',
               patient_name="MedSyn 3",
               patient_id="2_3",
               description=description,

               )
add_metadata_to_series('2.3.3', prompt, 'SeriesPrompt')


In [ ]:
prompt="No bullae, cystic lung disease, or CT findings of small airways disease."
description="no bullae"
nifti_file = '../data/nifti/MedSynExamples/no_bullae_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.4',
               series_instance_uid='2.4.1',
               study_id='1',
               patient_name="MedSyn 4",
               patient_id="2_4",
               description=description,

               )
add_metadata_to_series('2.4.1', prompt, 'SeriesPrompt')

In [ ]:
prompt="Emphysema is present with bullae. No cystic lung disease, or CT findings of small airways disease."
description="with bullae"
nifti_file = '../data/nifti/MedSynExamples/with_bullae_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.5',
               series_instance_uid='2.5.1',
               study_id='1',
               patient_name="MedSyn 5",
               patient_id="2_5",
               description=description,

               )
add_metadata_to_series('2.5.1', prompt, 'SeriesPrompt')

In [ ]:
prompt="There is no significant mediastinal lymphadenopathy. There is moderate cardiomegaly. The visualized upper abdominal organs are unremarkable. There is minimal perihepatic free fluid."
description="with cardiomegaly v2"
nifti_file = '../data/nifti/MedSynExamples/with_cardiomegaly_v2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.6',
               series_instance_uid='2.6.1',
               study_id='1',
               patient_name="MedSyn 6",
               patient_id="2_6",
               description=description,

               )
add_metadata_to_series('2.6.1', prompt, 'SeriesPrompt')

In [ ]:
prompt="Lungs: There is pulmonary fibrosis present with honeycombing demonstrated involving a significant portion of the bilateral lower lung zones. There are small areas of honeycombing in the anterior lateral right lung apex. There is no focal consolidation or pulmonary mass lesion demonstrated. There are a few areas of groundglass opacity in the right lung present, but the major finding is the pulmonary fibrosis that is most significant in the basilar regions. Pleural spaces:  There are small bilateral pleural effusions. There are no areas of significant pleural thickening demonstrated. Mediastinum and Lymph Nodes:  There is no pathologic adenopathy demonstrated in the axilla, mediastinum, or hilar regions. Heart and vascular structures: There is cardiomegaly. There is a cardiac pacemaker in place. There are small areas of atherosclerotic disease present, likely to include in the coronary arteries. There is no aneurysmal dilatation of the thoracic aorta. There is no significant pericardial effusion. Esophagus and visualized portion of the gastrointestinal tract: There is a mildly dilated partially fluid-filled distal esophagus. This could represent a hiatal hernia. Osseous structures and chest wall:  Unremarkable without acute or significant non-degenerative abnormalities. Visualized portion of the lower neck:  No major abnormalities are demonstrated in the portion of the neck included on this chest CT scan. Visualized portion of the upper abdomen:  There is ascites present with a small amount of fluid surrounding the liver and spleen. There is significant pulmonary fibrosis present, with large areas of honeycombing present involving both lower lung zones. There are milder areas of abnormality in the subpleural right greater than left upper lung zones. There are mild areas of groundglass opacity in the right upper lung zone, but there is no consolidation in the lungs. There are no suspicious pulmonary mass lesions. There are very small bilateral pleural effusions. There is at least mild ascites seen in the portion of the upper abdomen included on this exam."
description="with cardiomegaly v3"
nifti_file = '../data/nifti/MedSynExamples/with_cardiomegaly_v3_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='2.7',
               series_instance_uid='2.7.1',
               study_id='1',
               patient_name="MedSyn 7",
               patient_id="2_7",
               description=description,

               )
add_metadata_to_series('2.7.1', prompt, 'SeriesPrompt')

# Pilot Study

## Radiopedia

In [11]:
findings = "Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta.  No occlusive pathology was detected in the trachea and both main bronchi. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Arterial wall calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/train/train_1001_a_1.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='3.1',
               series_instance_uid='3.1.1',
               study_id='1',
               patient_name="Study Example 1",
               patient_id="train_1001_a_1_1",
               description= description,
               )
add_metadata_to_study('3.1', findings, 'Findings')
add_metadata_to_study('3.1', impressions, 'Impressions')

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_1001_a_1


C:\Users\mot99\AppData\Roaming\Python\Python312\site-packages\pydicom\valuerep.py:443: UserWarning: The value length (94) exceeds the maximum length of 64 allowed for VR LO.
  warnings.warn(msg)


Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost


In [12]:
prompt= 'Cardiomegaly, minimal pericardial-pleural effusion.'
description=""
nifti_file = '../data/nifti/examples/3_1_2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Cardiomegaly (generated)", 
               modality = "AI",
               study_instance_uid='3.1',
               series_instance_uid='3.1.2',
               study_id='2',
               patient_name="Study Example 1",
               patient_id="train_1001_a_1_1",
               description=description,

               )
add_metadata_to_series('3.1.2', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/examples/3_1_2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/3_1_2_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
91c9b0af-f1c4b220-6a7e72b1-2bcef106-b4033d31


In [13]:
prompt= 'Consolidation areas in both lung lower lobes prominent.'
description=""
nifti_file = '../data/nifti/examples/3_1_3_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Consolidation (generated)", 
               modality = "AI",
               study_instance_uid='3.1',
               series_instance_uid='3.1.3',
               study_id='3',
               patient_name="Study Example 1",
               patient_id="train_1001_a_1_1",
               description=description,

               )
add_metadata_to_series('3.1.3', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/examples/3_1_3_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/3_1_3_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
b560f7a6-dcc0952f-e8031c93-4721622a-39028892


In [14]:
prompt= 'Atelectasis in the lingular segment of the left lung.'
description=""
nifti_file = '../data/nifti/examples/3_1_4_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Atelectasis (generated)", 
               modality = "AI",
               study_instance_uid='3.1',
               series_instance_uid='3.1.4',
               study_id='4',
               patient_name="Study Example 1",
               patient_id="train_1001_a_1_1",
               description=description,

               )
add_metadata_to_series('3.1.4', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/examples/3_1_4_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/3_1_4_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
b645679c-2c0a23ae-8b87aa3a-d3801448-ba4ba0ce


In [15]:
prompt="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description=""
nifti_file = '../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Generated, 1", 
               modality = "AI",
               study_instance_uid='3.1',
               series_instance_uid='3.1.5',
               study_id='5',
               patient_name="Study Example 1",
               patient_id="train_1001_a_1_1",
               description=description,
               )
add_metadata_to_series('3.1.5', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/img_256_standard/train_1001_a_1_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
826180be-cff708dd-69d6a2d8-9a4e9d0a-72fc416e


## Counterfactual

In [16]:
findings = "Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta.  No occlusive pathology was detected in the trachea and both main bronchi. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Arterial wall calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/train/train_1001_a_1.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='3.2',
               series_instance_uid='3.2.1',
               study_id='1',
               patient_name="Study Example 2",
               patient_id="train_1001_a_1_2",
               description= description,
               )
add_metadata_to_study('3.2', findings, 'Findings')
add_metadata_to_study('3.2', impressions, 'Impressions')

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_1001_a_1
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost


In [17]:
prompt= 'Cardiomegaly, minimal pericardial-pleural effusion.'
description=""
nifti_file = '../data/nifti/examples/3_2_2_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Minimal Pericardial Effusion (generated)", 
               modality = "AI",
               study_instance_uid='3.2',
               series_instance_uid='3.2.2',
               study_id='2',
               patient_name="Study Example 2",
               patient_id="train_1001_a_1_2",
               description=description,

               )
add_metadata_to_series('3.2.2', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/examples/3_2_2_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/3_2_2_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
27727bf5-96263f3d-e41128cd-bbdc7464-a7c33514


In [18]:
# Haluzination
prompt= 'Cardiomegaly, large pericardial-pleural effusion.'
description=""
nifti_file = '../data/nifti/examples/3_2_3_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Large Pericardial Effusion (generated)", 
               modality = "AI",
               study_instance_uid='3.2',
               series_instance_uid='3.2.3',
               study_id='3',
               patient_name="Study Example 2",
               patient_id="train_1001_a_1_2",
               description=description,
               )
add_metadata_to_series('3.2.3', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/examples/3_2_3_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/3_2_3_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
c99533fe-32742974-cdf18ec7-4e226aad-220260c6


In [19]:
prompt= 'Cardiomegaly, no pericardial-pleural effusion.'
description=""
nifti_file = '../data/nifti/examples/3_2_4_sample_0.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="No Pericardial Effusion (generated)", 
               modality = "AI",
               study_instance_uid='3.2',
               series_instance_uid='3.2.4',
               study_id='4',
               patient_name="Study Example 2",
               patient_id="train_1001_a_1_2",
               description=description,

               )
add_metadata_to_series('3.2.4', prompt, 'SeriesPrompt')

Store DICOM files in Folder: ../data/nifti/examples/3_2_4_sample_0
Conversion complete. DICOM files are saved in ../data/nifti/examples/3_2_4_sample_0
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
661f8b59-2fb2500c-690ee75c-46b43fd8-ed852519


## Empty Example

In [20]:
findings = "Minimal pericardial and bilateral pleural effusion are observed. Millimetric calcific plaques follow in the aorta.  No occlusive pathology was detected in the trachea and both main bronchi. There is an area of atelectasis accompanied by volume loss in the lingular segment of the left lung upper lobe. There are significant increases in interlobular septal thickness in the lower lobes of both lungs on the left. No mass was detected in both lungs. No pathological wall thickness increase was observed in the esophagus within the sections. No lytic-destructive lesions were detected in the bone structures within the sections."
impressions="Cardiomegaly, minimal pericardial-pleural effusion. Consolidation areas in both lung lower lobes prominent on the left; is regressed. Atelectasis in the lingular segment of the left lung"
description= "Arterial wall calcification, Cardiomegaly, Pericardial effusion, Atelectasis, Pleural effusion"
nifti_file = '../data/nifti/ct-rate/train/train_1001_a_1.nii.gz'

nifti_to_dicom(nifti_file=nifti_file, 
               series_description="Original", 
               modality = "CT",
               study_instance_uid='3.3',
               series_instance_uid='3.3.1',
               study_id='1',
               patient_name="Study Example 3",
               patient_id="train_1001_a_1_3",
               description= description,
               )
add_metadata_to_study('3.3', findings, 'Findings')
add_metadata_to_study('3.3', impressions, 'Impressions')

Store DICOM files in Folder: ../data/nifti/ct-rate/train/train_1001_a_1
Conversion complete. DICOM files are saved in ../data/nifti/ct-rate/train/train_1001_a_1
Uploading DICOM folder to Orthanc...
Files Uploaded to Orthanc Server Localhost
